<a href="https://colab.research.google.com/github/Tomawock/NLP_Attack/blob/main/models/studio_2/DDI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install optuna 

In [1]:
import pickle
import pandas as pd
import optuna
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from tensorflow import keras
from sklearn.utils.class_weight import compute_class_weight

In [3]:
path="Desktop\\DDI"

In [ ]:
path="/content/drive/Shareddrives/Deep Learning/datasets/DDI/"

PATH PER WINDOWDS



In [ ]:
dev = pd.read_csv(path+'\ddi2013-type\\dev.tsv', sep='\t')
test = pd.read_csv(path+'\ddi2013-type\\test.tsv', sep='\t')
train = pd.read_csv(path+'\ddi2013-type\\train.tsv', sep='\t')

data_sinonimi = pd.read_csv(path+"\ddi2013-type\\DDI_sinonimi_test.csv")
data_embedding = pd.read_csv(path+"\ddi2013-type\\DDI_embedding_test.csv")

data_embedding_train = pd.read_csv(path+"\ddi2013-type\\ .csv")

print(test.shape)
print(data_sinonimi.shape)
print(test)

In [ ]:
dev = pd.read_csv(path+'ddi2013-type/dev.tsv', sep='\t')
test = pd.read_csv(path+'ddi2013-type/test.tsv', sep='\t')
train = pd.read_csv(path+'ddi2013-type/train.tsv', sep='\t')

data_sinonimi = pd.read_csv(path+"ddi2013-type/DDI_sinonimi_test.csv")
data_embedding = pd.read_csv(path+"ddi2013-type/DDI_embedding_test.csv")

data_embedding_train = pd.read_csv(path+"ddi2013-type/DDI_embedding.csv")

print(test.shape)
print(data_sinonimi.shape)
print(test)

In [7]:
test_study_1 = pd.concat([test, data_embedding], ignore_index=True)
train_study_1 = pd.concat([train, data_embedding_train], ignore_index=True)
print (test_study_1.shape)
print (train_study_1.shape)

(11522, 3)
(37558, 3)


In [ ]:
print(train.shape)
print(data_sinonimi.shape)
print(data_embedding.shape)

In [ ]:
print(train.label.value_counts())
print(data_sinonimi.label.value_counts())
print(data_embedding.label.value_counts())

In [8]:
with open(path+"\word2index.pkl", 'rb') as output:
  w2i = pickle.load(output)
with open(path+"\embedding_matrix.pkl", 'rb') as output:
  embedding_matrix = pickle.load(output)

Set up per allenamento Modello

In [9]:
categories = [['DDI-false', 'DDI-mechanism', 'DDI-effect', 'DDI-advise','DDI-int']]

my_text_to_word_sequence = lambda sen: keras.preprocessing.text.text_to_word_sequence(sen,
                                                                                      filters='!"#&()*+,-./:;<=>?[\\]^_`\'{|}~\t\n',
                                                                                      lower=True)

TRAINSET

In [10]:
five_hot_train = OneHotEncoder(sparse=False, categories=categories).fit_transform(
  train_study_1.label.to_numpy().reshape(-1, 1)
)

In [11]:
sentences_train = [my_text_to_word_sequence(sentence) for sentence in train_study_1['sentence']]

TESTSET

In [12]:
five_hot_test = OneHotEncoder(sparse=False, categories=categories).fit_transform(
  test_study_1.label.to_numpy().reshape(-1, 1)
)

In [13]:
sentences_test = [my_text_to_word_sequence(sentence) for sentence in test_study_1['sentence']]

ORIGINALE

In [14]:
five_hot_orig = OneHotEncoder(sparse=False, categories=categories).fit_transform(
  test.label.to_numpy().reshape(-1, 1)
)

In [15]:
sentences_orig = [my_text_to_word_sequence(sentence) for sentence in test['sentence']]

SINONIMI

In [16]:
five_hot_sin = OneHotEncoder(sparse=False, categories=categories).fit_transform(
  data_sinonimi.label.to_numpy().reshape(-1, 1)
)

In [17]:
sentences_sin = [my_text_to_word_sequence(sentence) for sentence in data_sinonimi['sentence']]

EMBEDDING

In [18]:
five_hot_emb = OneHotEncoder(sparse=False, categories=categories).fit_transform(
  data_embedding.label.to_numpy().reshape(-1, 1)
)

In [19]:
sentences_emb = [my_text_to_word_sequence(sentence) for sentence in data_embedding['sentence']]

Estrai la massima dimensione dell'input in base ai vari dataset considerati

In [ ]:
max_index, max = (-1, -1)
for i, sentence in enumerate(sentences_train):
  max_index, max = (i, len(sentence)) if len(sentence) > max else (max_index, max)
for i, sentence in enumerate(sentences_test):
  max_index, max = (i, len(sentence)) if len(sentence) > max else (max_index, max)
for i, sentence in enumerate(sentences_orig):
  max_index, max = (i, len(sentence)) if len(sentence) > max else (max_index, max)
for i, sentence in enumerate(sentences_sin):
  max_index, max = (i, len(sentence)) if len(sentence) > max else (max_index, max)
for i, sentence in enumerate(sentences_emb):
  max_index, max = (i, len(sentence)) if len(sentence) > max else (max_index, max)
  
print(f'Il massimo è {max}')

Crao i vari embedding per tutti i dataset, quest'operazione e pesante 

In [21]:
embedded_trainset = np.zeros(shape=(len(sentences_train), max, 300))
for i, sentence in enumerate(sentences_train):
  for j, word in enumerate(sentence):
    try:
      embedded_trainset[i, j, :] = embedding_matrix[w2i[word]]
    except KeyError:
      pass

In [22]:
embedded_origin = np.zeros(shape=(len(sentences_orig), max, 300))
for i, sentence in enumerate(sentences_orig):
  for j, word in enumerate(sentence):
    try:
      embedded_origin[i, j, :] = embedding_matrix[w2i[word]]
    except KeyError:
      pass

In [23]:
embedded_testset = np.zeros(shape=(len(sentences_test), max, 300))
for i, sentence in enumerate(sentences_test):
  for j, word in enumerate(sentence):
    try:
      embedded_testset[i, j, :] = embedding_matrix[w2i[word]]
    except KeyError:
      pass

In [24]:
embedded_sin = np.zeros(shape=(len(sentences_sin), max, 300))
for i, sentence in enumerate(sentences_sin):
  for j, word in enumerate(sentence):
    try:
      embedded_sin[i, j, :] = embedding_matrix[w2i[word]]
    except KeyError:
      pass

In [25]:
embedded_emb = np.zeros(shape=(len(sentences_emb), max, 300))
for i, sentence in enumerate(sentences_emb):
  for j, word in enumerate(sentence):
    try:
      embedded_emb[i, j, :] = embedding_matrix[w2i[word]]
    except KeyError:
      pass

Carica optuna results e inizializza il modello, oppure salva il modello oppure carica solo i pesi del modello 

In [26]:
best_params = optuna.load_study(study_name="DDI", storage="sqlite:///"+path+"\ddi2013-type\\optuna_ddi_studio_0.db").best_params

In [ ]:
print(f'{best_params}')

In [28]:
model = keras.Sequential()
model.add(keras.layers.Input(shape=(max, 300)))
model.add(keras.layers.Bidirectional(layer=keras.layers.LSTM(units=best_params['units'],
                                                             recurrent_dropout=best_params['dropout'],
                                                             activation='tanh')))

model.add(keras.layers.Dense(5, activation='softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer=keras.optimizers.Adam(learning_rate=0.001),
              metrics=['accuracy'])

In [29]:
#train the model
result = model.fit(embedded_trainset,
                   five_hot_train,
                   epochs=100,
                   batch_size=best_params['batch_size'],
                   callbacks=[keras.callbacks.EarlyStopping(monitor='loss',
                                                            patience=10)])

Epoch 1/100
422/422 [==============================] - 90s 213ms/step - loss: 0.5936 - accuracy: 0.8417
Epoch 2/100
422/422 [==============================] - 93s 220ms/step - loss: 0.4776 - accuracy: 0.8455
Epoch 3/100
422/422 [==============================] - 98s 232ms/step - loss: 0.4275 - accuracy: 0.8476
Epoch 4/100
422/422 [==============================] - 95s 224ms/step - loss: 0.3983 - accuracy: 0.8491
Epoch 5/100
422/422 [==============================] - 92s 218ms/step - loss: 0.3792 - accuracy: 0.8525
Epoch 6/100
422/422 [==============================] - 93s 220ms/step - loss: 0.3637 - accuracy: 0.8557
Epoch 7/100
422/422 [==============================] - 94s 222ms/step - loss: 0.3491 - accuracy: 0.8592
Epoch 8/100
422/422 [==============================] - 91s 216ms/step - loss: 0.3367 - accuracy: 0.8621
Epoch 9/100
422/422 [==============================] - 93s 220ms/step - loss: 0.3241 - accuracy: 0.8669
Epoch 10/100
422/422 [==============================] - 93s 220m

In [30]:
#save the model
model.save_weights(path+'\ddi2013-type\\DDI_w_studio_2.h5')

In [ ]:
#load only the w of the model, the model must be already executed
model.load_weights(path+'\ddi2013-type\\DDI_w_studio_2.h5')

EVALUATE ALL DATASET

In [31]:
result_base=model.evaluate(embedded_trainset, five_hot_train, batch_size=best_params['batch_size'],)
print(f'DATASET ORIGINARIO{result_base}')

422/422 [==============================] - 22s 52ms/step - loss: 0.0281 - accuracy: 0.9905
DATASET ORIGINARIO[0.02805744670331478, 0.9905479550361633]


In [32]:
result_base=model.evaluate(embedded_testset, five_hot_test, batch_size=best_params['batch_size'],)
print(f'DATASET TEST{result_base}')

130/130 [==============================] - 7s 56ms/step - loss: 0.8106 - accuracy: 0.8373
DATASET TEST[0.810559868812561, 0.8372678160667419]


In [33]:
result_base=model.evaluate(embedded_origin, five_hot_orig, batch_size=best_params['batch_size'],)
print(f'DATASET TEST{result_base}')

65/65 [==============================] - 4s 56ms/step - loss: 0.7490 - accuracy: 0.8441
DATASET TEST[0.7489575743675232, 0.8441242575645447]


In [34]:
result_base=model.evaluate(embedded_sin, five_hot_sin, batch_size=best_params['batch_size'],)
print(f'DATASET SINONIMI{result_base}')

65/65 [==============================] - 4s 55ms/step - loss: 1.0974 - accuracy: 0.8292
DATASET SINONIMI[1.0973966121673584, 0.8291963338851929]


In [35]:
result_base=model.evaluate(embedded_emb, five_hot_emb, batch_size=best_params['batch_size'],)
print(f'DATASET EMBEDDING{result_base}')

65/65 [==============================] - 4s 55ms/step - loss: 0.8722 - accuracy: 0.8304
DATASET EMBEDDING[0.8721619248390198, 0.8304113745689392]


tali test sono stati effettuati su windows, eventuali path necessitano di essere adattati, se effettuati su sistema operativo unix like.

